In [1]:
import numpy as np
import GPy

Problem?\
1-task ICM does not perform as well as SOGP in BOSS MT\
Solution?\
Fix hyperparameters so that $\bf{B} = [var]$

# Covariance functions

## RBF

$k_{RBF}(x,x') = \sigma^2e^{-(x-x')^2/2L^2}$

where\
$\sigma^2 = $ variance, scaling factor\
$L = $ lenghtscale, 'smoothness': large $L$ = smooth curve

## STDP

$k_{STDP}(x,x') = \sigma^2e^{-2\text{sin}^2(\pi|x-x'|)/pL^2}$

where\
$p = $ period

covariance functions may also be more complex, or they can be added or multiplied together for example to add support for multiple dimensions.


# SOGP kernel

$\bf{K} = \begin{pmatrix}
    k(\textbf{x}_1,\textbf{x}_1) & \dots & k(\textbf{x}_1,\textbf{x}_N) \\
    \vdots & \ddots & \vdots \\
    k(\textbf{x}_N,\textbf{x}_1) & \dots & k(\textbf{x}_N,\textbf{x}_N) \\
    \end{pmatrix}$,

where $N = $ number of inputs and
$k(\bf{x},\bf{x}') = $ valid covariance function for vector input $\bf{x} \in \mathbf{R}^d$

# Coregionalization kernels

Kernel and coregionalization matrix produce MT kernel.\
Now coregionalization matrix consumes the variance $\sigma^2$, so it is fixed to one in the covariance functions.

## LCM kernel

$\bf{K}_{LMC} = \sum_{q=1}^{Q}\bf{B}_q\otimes\bf{K}_q$ (see Kronecker product below)

## ICM kernel (Q = 1)

$\bf{K}_{IMC} = \bf{B}_1\otimes\bf{K}_1$

### Coregionalization matrix $\bf{B}_q$:
$\bf{B}_q = \bf{W}_q\bf{W}_q^\top$\
or (how it is in GPy, I have not yet found original source to this, althought it makes sense)\
$\bf{B}_q = \bf{W}_q\bf{W}_q^\top + diag(\bf{\kappa})$

where:\
$\bf{W}_q \in \mathbf{R}^{d\times r}$ encodes correlation between tasks,\
$\bf{\kappa} \in \mathbf{R}^r$ allows independence (or variance) of tasks, and\
$r \in \{1,...,num\_tasks\}$ is a grouping factor that may be used to group tasks that share covariance functions

### Kronecker product $\otimes$

$\begin{pmatrix} a & b \\ c & d\end{pmatrix} \otimes \begin{pmatrix} 1 & 2 \\ 3 & 4 \end{pmatrix} = \begin{pmatrix}  a1 & a2 & b1 & b2 \\ a3 & a4 & b3 & b4 \\ c1 & c2 & d1 & d2 \\ c3 & c4 & d3 & d4 \end{pmatrix}$

In [11]:
W = np.array([1,2])
k = np.array([4,1])
W.dot(A.transpose())+np.diag(k)

array([[9, 5],
       [5, 6]])

# GPr

Prior predictive for new observations:

\begin{equation}p(y_*,\textbf{y}|\textbf{x}_*,\textbf{X})
    \sim
    N\begin{pmatrix}
    \begin{matrix}
    m(\textbf{x}_1) \\ \vdots \\ m(\textbf{x}_j) \\ m(\textbf{x}_*)
    \end{matrix}
    ,
    \begin{matrix}
        \begin{pmatrix}
        k(\textbf{x}_1,\textbf{x}_1) & \dots & k(\textbf{x}_1,\textbf{x}_j) \\
        \vdots & \ddots & \vdots \\
        k(\textbf{x}_j,\textbf{x}_1) & \dots & k(\textbf{x}_j,\textbf{x}_j) \\
        \end{pmatrix}
        \begin{pmatrix}
        k(\textbf{x}_1,\textbf{x}_*) \\ \vdots \\ k(\textbf{x}_j,\textbf{x}_*)
        \end{pmatrix}
        \\
        \begin{pmatrix}
        k(\textbf{x}_*,\textbf{x}_1) & \dots & k(\textbf{x}_*,\textbf{x}_j) \\
        \end{pmatrix}
        \begin{pmatrix}
        k(\textbf{x}_*,\textbf{x}_*)
        \end{pmatrix}
    \end{matrix}
    \end{pmatrix}
    =
    N\begin{pmatrix}
    \begin{vmatrix}
    \bf{\mu} \\ \mu_*
    \end{vmatrix}
    ,
    \begin{vmatrix}
        \mathbf{K}
        & 
        \mathbf{K_*^\top}
        \\
        \mathbf{K_*}
        &
        \mathbf{K_{**}}
    \end{vmatrix}
    \end{pmatrix}
    .
\end{equation}

Posterior predictive:\
\begin{equation}
     p(y_*|\textbf{x}_*,\textbf{X},\textbf{y}) \sim N(\hat\mu,\hat\nu).
\end{equation}

The distribution parameters can be solved from:

\begin{equation}
\label{muhatsolution}
    \hat\mu = \mathbf{K}_*^\top\mathbf{K}^{-1}(\textbf{y}-\bf{\mu})+\mu_*
\end{equation}
and
\begin{equation}
\label{nuhatsolution}
    \hat\nu = \mathbf{K}_*^\top\mathbf{K}^{-1}\mathbf{K}_* + \mathbf{K}_{**}.
\end{equation}

## BOSS
Covariance function:

Hyperparameter priors:
$d = y_{max}-y_{min}$\
$var = gammadist(2, 2/(d/2)^2)$\
$lengthscales = [gammadist(3.3678, 9.0204)_1, ..., gammadist(3.3678, 9.0204)_{dim}]$


In [7]:
### STS thetainit
# default initial hyperparameters
        if self.thetainit is None:
            self.thetainit = [0.5 * (self.yrange[1] - self.yrange[0])] # sig
            for i in range(self.dim): # lengthscales
                if self.kerntype[i] == 'stdp': # pbc
                    self.thetainit.append(np.pi/10)
                else: # nonpbc
                    self.thetainit.append(self.periods[i]/20)


### kernelfactory
for i in range(STS.dim):
            if forced_hypers is None:
                if i == 0:
                    Ksi = STS.thetainit[0]
                else:
                    Ksi = 1.0
                Klsi = STS.thetainit[i + 1]
            else:
                Ksi = forced_hypers[0]
                Klsi = forced_hypers[1 + i]
            Kp = STS.periods[i]

            if STS.kerntype[i] == "stdp":
                kerns[i] = GPy.kern.StdPeriodic( ##########################
                    input_dim=1,
                    variance=Ksi,
                    period=Kp,
                    lengthscale=Klsi,
                    ARD1=True,
                    ARD2=True,
                    active_dims=[i],
                    name="kern",
                )
                
### GPy stdperiodic
def __init__(self, input_dim, variance=1., period=None, lengthscale=None, ARD1=False, ARD2=False, active_dims=None, name='std_periodic',useGPU=False):
        super(StdPeriodic, self).__init__(input_dim, active_dims, name, useGPU=useGPU)
        self.ARD1 = ARD1 # correspond to periods
        self.ARD2 = ARD2 # correspond to lengthscales

        self.name = name

        if self.ARD1 == False:
            if period is not None: #### shared
                period = np.asarray(period)
                assert period.size == 1, "Only one period needed for non-ARD kernel"
            else:
                period = np.ones(1)
        else:
            if period is not None:
                period = np.asarray(period)
                assert period.size == input_dim, "bad number of periods"
            else:
                period = np.ones(input_dim)

        if self.ARD2 == False: ##### shared
            if lengthscale is not None:
                lengthscale = np.asarray(lengthscale)
                assert lengthscale.size == 1, "Only one lengthscale needed for non-ARD kernel"
            else:
                lengthscale = np.ones(1)
        else:
            if lengthscale is not None:
                lengthscale = np.asarray(lengthscale)
                assert lengthscale.size == input_dim, "bad number of lengthscales"
            else:
                lengthscale = np.ones(input_dim)

        self.variance = Param('variance', variance, Logexp()) ############### variance
        assert self.variance.size==1, "Variance size must be one"
        self.period =  Param('period', period, Logexp())
        self.lengthscale =  Param('lengthscale', lengthscale, Logexp())

        self.link_parameters(self.variance,  self.period, self.lengthscale)


array([[1, 0],
       [0, 2]])

## BOSS-MT (currently)
GPy defaults




In [ ]:
#-- default hyperparameter priors
        if self.thetapriorpar is None and self.thetaprior is not None:
            if self.thetaprior == "gamma":
                self.thetapriorpar = [[1,1]] # not used, but must be passed to kernelfactory initialized
            else:
                raise TypeError("Unknown options set for thetaprior: '"
                                + self.thetaprior + "'.")

            for i in range(self.dim):
                if self.thetaprior == "gamma":
                    if self.kerntype[i] == 'stdp': # pbc
                        shape = 3.3678
                        rate = 9.0204
                    else: # nonpbc
                        shape, rate = Distributions.gammaparams(
                            self.periods[i]/20, self.periods[i]/2
                                                 )
                    self.thetapriorpar.append([shape, rate])
                else:
                    raise TypeError("Unknown options set for \
                                    K_priortype: '" + self.thetaprior + "'.")

# default initial hyperparameters
        if self.thetainit is None:
            self.thetainit = [1] # var fixed, change kernel constructer later so that it does not need
            for i in range(self.dim): # lengthscales
                if self.kerntype[i] == 'stdp': # pbc
                    self.thetainit.append(np.pi/10)
                else: # nonpbc
                    self.thetainit.append(self.periods[i]/20)


### BOSS MT
self.kernel = GPy.util.multioutput.LCM(dim, num_tasks, kernels_list, W_rank)
        # set kernel variance fixed to 1 (gpy sets the same variance for all kernels)
self.kernel.kern.variance.constrain_fixed(1)

### multioutput
def ICM(input_dim, num_outputs, kernel, W_rank=1,W=None,kappa=None,name='ICM'):
    """
    Builds a kernel for an Intrinsic Coregionalization Model

    :input_dim: Input dimensionality (does not include dimension of indices)
    :num_outputs: Number of outputs
    :param kernel: kernel that will be multiplied by the coregionalize kernel (matrix B).
    :type kernel: a GPy kernel
    :param W_rank: number tuples of the corregionalization parameters 'W'
    :type W_rank: integer
    """
    if kernel.input_dim != input_dim:
        kernel.input_dim = input_dim
        warnings.warn("kernel's input dimension overwritten to fit input_dim parameter.")

    K = kernel.prod(GPy.kern.Coregionalize(1, num_outputs, active_dims=[input_dim], rank=W_rank,W=W,kappa=kappa,name='B'),name=name)
    return K
def LCM(input_dim, num_outputs, kernels_list, W_rank=1,name='ICM'):
    """
    Builds a kernel for an Linear Coregionalization Model

    :input_dim: Input dimensionality (does not include dimension of indices)
    :num_outputs: Number of outputs
    :param kernel: kernel that will be multiplied by the coregionalize kernel (matrix B).
    :type kernel: a GPy kernel
    :param W_rank: number tuples of the corregionalization parameters 'W'
    :type W_rank: integer
    """
    Nk = len(kernels_list)
    K = ICM(input_dim,num_outputs,kernels_list[0],W_rank,name='%s%s' %(name,0))
    j = 1
    for kernel in kernels_list[1:]:
        K += ICM(input_dim,num_outputs,kernel,W_rank,name='%s%s' %(name,j))
        j += 1
    return K
### coregionalize

def __init__(self, input_dim, output_dim, rank=1, W=None, kappa=None, active_dims=None, name='coregion'):
        super(Coregionalize, self).__init__(input_dim, active_dims, name=name)
        self.output_dim = output_dim
        self.rank = rank
        if self.rank>output_dim:
            print("Warning: Unusual choice of rank, it should normally be less than the output_dim.")
        if W is None:
            W = 0.5*np.random.randn(self.output_dim, self.rank)/np.sqrt(self.rank)
        else:
            assert W.shape==(self.output_dim, self.rank)
        self.W = Param('W', W)
        if kappa is None:
            kappa = 0.5*np.ones(self.output_dim)
        else:
            assert kappa.shape==(self.output_dim, )
        self.kappa = Param('kappa', kappa, Logexp())
        self.link_parameters(self.W, self.kappa)

[docs]    def parameters_changed(self):
        self.B = np.dot(self.W, self.W.T) + np.diag(self.kappa)

## BOSS-MT (as should)